## Deliverable 1

We will follow the CRISP-DM methodology

### Business Understanding

The goal of this project is to create a regression model that can predict car prices based on its details. This includes:
- Regression Benchmarking
- Model Optimization
- Additional Insights

### Data Understanding

#### **Metadata**:

- **carID** :  An attribute that contains an identifier for each car.
- **Brand** :  The car’s main brand (e.g. Ford, Toyota).
- **model** : The car model.
- **year** : The year of Registration of the Car.
- **mileage** : The total reported distance travelled by the car (in miles).
- **tax** :  The amount of road tax (in £) that, in 2020, was applicable to the car in question. 
- **fuelType** :  Type of Fuel used by the car (Diesel, Petrol, Hybrid, Electric).
- **mpg** : Average Miles per Gallon.
- **engineSize** : Size of Engine in liters (Cubic Decimeters).
- **paintQuality%** :  The mechanic’s assessment of the cars’ overall paint quality and hull integrity (filled by the mechanic during evaluation). 
- **previousOwners** : Number of previous registered owners of the vehicle.
- **hasDamage** :  Boolean marker filled by the seller at the time of registration stating whether the car is damaged or not.
- **price** : The car’s price when purchased by Cars 4 You (in £).


In [14]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [15]:
# Import the train and test data sets
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [16]:
df_train.head()

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
0,69512,VW,Golf,2016.0,22290,Semi-Auto,28421.0,Petrol,NaN,11.417268,2.0,63.0,4.000000,0.0
1,53000,Toyota,Yaris,2019.0,13790,Manual,4589.0,Petrol,145.0,47.900000,1.5,50.0,1.000000,0.0
2,6366,Audi,Q2,2019.0,24990,Semi-Auto,3624.0,Petrol,145.0,40.900000,1.5,56.0,4.000000,0.0
3,29021,Ford,FIESTA,2018.0,12500,anual,9102.0,Petrol,145.0,65.700000,1.0,50.0,-2.340306,0.0
4,10062,BMW,2 Series,2019.0,22995,Manual,1000.0,Petrol,145.0,42.800000,1.5,97.0,3.000000,0.0


We can already see that we have Null Values (at least in tax) and have strange values like negative previous owners

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75973 entries, 0 to 75972
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   carID           75973 non-null  int64  
 1   Brand           74452 non-null  object 
 2   model           74456 non-null  object 
 3   year            74482 non-null  float64
 4   price           75973 non-null  int64  
 5   transmission    74451 non-null  object 
 6   mileage         74510 non-null  float64
 7   fuelType        74462 non-null  object 
 8   tax             68069 non-null  float64
 9   mpg             68047 non-null  float64
 10  engineSize      74457 non-null  float64
 11  paintQuality%   74449 non-null  float64
 12  previousOwners  74423 non-null  float64
 13  hasDamage       74425 non-null  float64
dtypes: float64(8), int64(2), object(4)
memory usage: 8.1+ MB


Identified Problems:

- Year and previousOwners is float when it should be integer
- hasDamage should be boolean instead of float

In [ ]:
df_train['']

In [28]:
features_to_check_values = ['year', 'previousOwners', 'hasDamage', 'Brand', 'transmission', 'fuelType']

for feat in features_to_check_values:
    print(f'{feat} :' )
    print(df_train[feat].unique())

year :
[2016.         2019.         2018.         2014.         2017.
 2020.         2013.                   nan 2015.         2023.36707842
 2011.         2012.         2023.1169636  2003.         2009.
 2007.         2005.         2011.11118842 2011.21085349 2023.97731126
 2004.         2010.         2010.56500919 2008.         2024.12175905
 2006.         2023.60527574 2023.38982198 2010.67696784 2001.
 2000.         2023.26798867 2010.26863473 2010.37154646 2002.
 2022.69668507 2009.81675711 2010.7464032  2012.69574039 1996.
 1998.         2022.87800554 1970.         1999.         1997.        ]
previousOwners :
[ 4.          1.         -2.34030622  3.          0.          2.
         nan -2.34565     6.25837066 -2.33512284  6.22789796  6.25823052
  6.23017958 -2.33936045  6.21772443  6.24583495 -2.29943868 -2.34010209
 -2.31225953  6.2482512  -2.31733109  6.23308217  6.24177863 -2.33744529]
hasDamage :
[ 0. nan]
Brand :
['VW' 'Toyota' 'Audi' 'Ford' 'BMW' 'Skoda' 'Opel' 'Mercedes' 

- The year column clearly has weird values with years such as 2023.26798867, we should round them using floor.
- Previous owners has negative values and floats, we should check the number of times they appear to see if they are probably simply a mistake
- For hasDamage we only have 0 and nan, in this case nan must mean yes or no.
- For Brand, Semi-Auto and fuelType we have a lot of variations of the same name, for example Toyota and toyot.

We will deal with this in the data preparation section


In [31]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
carID,75973.0,37986.000000,21931.660338,0.000000,18993.00,37986.0,56979.0,75972.000000
year,74482.0,2017.096611,2.208704,1970.000000,2016.00,2017.0,2019.0,2024.121759
price,75973.0,16881.889553,9736.926322,450.000000,10200.00,14699.0,20950.0,159999.000000
mileage,74510.0,23004.184088,22129.788366,-58540.574478,7423.25,17300.0,32427.5,323000.000000
tax,68069.0,120.329078,65.521176,-91.121630,125.00,145.0,145.0,580.000000
mpg,68047.0,55.152666,16.497837,-43.421768,46.30,54.3,62.8,470.800000
engineSize,74457.0,1.660136,0.573462,-0.103493,1.20,1.6,2.0,6.600000
paintQuality%,74449.0,64.590667,21.021065,1.638913,47.00,65.0,82.0,125.594308
previousOwners,74423.0,1.994580,1.472981,-2.345650,1.00,2.0,3.0,6.258371
hasDamage,74425.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000


The minimum in the year is 1970 which is ver far apart from the rest of the years, maybe it was an error.

We also have negative values for mileage, tax, mpg, engine_Size and previousOwners

paintQuality% is a percentage so it should not be over 100, but the max is 125.594308

From the  huge difference between Q3 and the max in a lot of features, we will probably have outliers.

In [30]:
df_train[ df_train['year'] == 1970]

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
11425,62732,OPEL,Zafira,1970.0,10495,Manual,37357.0,NaN,200.0,42.2,NaN,60.0,2.0,0.0
34917,35769,Mercedes,M Class,1970.0,24999,Automatic,14000.0,Diesel,305.0,39.2,0.0,44.0,3.0,0.0


We only have 2 observations with this year, it was most likely a mistake, we will either remove this observations or replace the value with something

In [32]:
df_train[ df_train['mileage'] < 0]

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
268,70615,VW,Tiguan,2020.0,25000,Manual,-48190.655673,Petrol,145.0,38.2,1.5,89.0,3.0,0.0
284,43529,Mercedes,C Class,2019.0,25780,Semi-Auto,-50755.210230,Diesel,NaN,NaN,2.0,72.0,3.0,0.0
325,71090,VW,Golf,2018.0,14995,Manual,-48190.655673,Diesel,150.0,57.7,1.6,83.0,2.0,0.0
843,7845,BMW,3 Series,2019.0,23498,Semi-Auto,-58540.574478,Diesel,145.0,54.3,2.0,38.0,NaN,0.0
853,24999,Ford,FIESTA,2017.0,8895,Manual,-42650.453719,Petrol,125.0,54.3,1.2,45.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74961,56373,Opel,NaN,2018.0,8514,Manual,-42707.564215,Petrol,150.0,55.4,1.4,49.0,1.0,NaN
75015,45765,Mercedes,GLE Class,2020.0,54995,Automatic,-50755.210230,Diesel,150.0,32.8,3.0,69.0,4.0,0.0
75423,38413,Mercedes,SLK,2015.0,17900,Automati,-50755.210230,Diesel,150.0,56.5,2.1,40.0,3.0,0.0
75859,55820,oyota,Aygo,2019.0,8491,Manual,-39765.602338,Petrol,145.0,56.5,1.0,56.0,0.0,0.0


We have 369 observations with negative mileage, its a lot to simply remove. Since it is a distance it doesnt mae any sense for it to be negative

In [33]:
df_train[ df_train['tax'] < 0]

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
26,35263,Mercedes,C Clas,NaN,11295,Automatic,45000.0,Petrol,-56.193202,46.3,1.6000,59.0,NaN,0.0
168,46204,Mercedes,C Class,2019.0,66699,Semi-Auto,630.0,Petrol,-56.193202,28.5,4.0000,32.0,2.0,0.0
360,24750,Ford,Fiesta,2019.0,17000,Manual,796.0,Petrol,-45.780610,58.9,1.0000,80.0,4.0,0.0
1051,20350,Ford,Focus,2015.0,7260,Manual,60661.0,Petrol,-45.780610,60.1,1.0000,NaN,2.0,0.0
1759,64978,Opel,Astra,2016.0,9595,Automatic,30765.0,Petrol,-20.241079,50.4,1.4000,30.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75111,11302,BMW,4 Series,2018.0,21470,Automatic,40553.0,Petrol,-43.242082,41.5,3.0000,44.0,3.0,0.0
75321,44238,Mercedes,SLK,2001.0,4990,Automatic,64476.0,Petrol,-56.193202,27.2,3.2000,47.0,3.0,0.0
75375,20309,Ford,Fiesta,2013.0,6000,Manual,48133.0,Petrol,-45.780610,54.3,2.6312,40.0,4.0,0.0
75543,33496,Hyundai,I20,2013.0,5495,Manual,30252.0,Petrol,-31.914945,57.6,1.2000,80.0,0.0,0.0


In [48]:
print(f' The number of observations with negative mileage is : {len(df_train[ df_train['mileage'] < 0])}')
print(f' The proportion of observations with negative mileage in the data set is : { round( (len(df_train[ df_train['mileage'] < 0]) / len(df_train))*100, 2)} %')


print()
print(f' The number of observations with negative tax is : {len(df_train[ df_train['tax'] < 0])}')
print(f' The proportion of observations with negative tax in the data set is : { round( (len(df_train[ df_train['tax'] < 0]) / len(df_train))*100, 2)} %')


print()
print(f' The number of observations with negative mpg is : {len(df_train[ df_train['mpg'] < 0])}')
print(f' The proportion of observations with negative mpg in the data set is : { round( (len(df_train[ df_train['mpg'] < 0]) / len(df_train))*100, 2)} %')


print()
print(f' The number of observations with negative engineSize is : {len(df_train[ df_train['engineSize'] < 0])}')
print(f' The proportion of observations with negative engineSize in the data set is : { round( (len(df_train[ df_train['engineSize'] < 0]) / len(df_train))*100, 2)} %')


print()
print(f' The number of observations with negative previousOwners is : {len(df_train[ df_train['previousOwners'] < 0])}')
print(f' The proportion of observations with negative previousOwners in the data set is : { round( (len(df_train[ df_train['previousOwners'] < 0]) / len(df_train))*100, 2)} %')

 The number of observations with negative mileage is : 369
 The proportion of observations with negative mileage in the data set is : 0.49 %

 The number of observations with negative tax is : 378
 The proportion of observations with negative tax in the data set is : 0.5 %

 The number of observations with negative mpg is : 36
 The proportion of observations with negative mpg in the data set is : 0.05 %

 The number of observations with negative engineSize is : 84
 The proportion of observations with negative engineSize in the data set is : 0.11 %

 The number of observations with negative previousOwners is : 371
 The proportion of observations with negative previousOwners in the data set is : 0.49 %


In [49]:
df_train[df_train['paintQuality%']>100]

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
171,15717,Ford,Fiesta,2018.0,10491,Manual,6522.0,Petrol,145.0,65.7,1.0,125.109951,3.0,0.0
194,11200,BMW,4 Series,2019.0,23591,Semi-Auto,6100.0,Petrol,145.0,45.6,2.0,125.569499,2.0,0.0
272,53403,Toyota,Aygo,2016.0,7995,Manual,25300.0,NaN,0.0,69.0,1.0,125.594308,1.0,0.0
467,48038,Skoda,Yeti Outdoor,2015.0,11995,Semi-Auto,19805.0,Petrol,125.0,51.4,1.2,125.453599,1.0,0.0
583,10137,BMW,5 Series,2017.0,21995,Automatic,18820.0,Diesel,145.0,65.7,2.0,125.569499,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74763,75768,VW,Tiguan,2018.0,18499,Manual,24940.0,Petrol,145.0,48.7,1.4,125.366507,0.0,0.0
75073,39666,Mercede,E Class,2017.0,21602,Semi-Auto,40690.0,Diesel,145.0,NaN,3.0,125.202033,3.0,0.0
75135,23930,Ford,Fiesta,2020.0,15000,Manual,35.0,Petrol,145.0,56.5,1.0,125.109951,0.0,0.0
75540,38756,Mercedes,GLS CLASS,2016.0,36495,Semi-Auto,33000.0,Diesel,300.0,37.2,3.0,125.202033,2.0,0.0


In [51]:
print(f' The number of observations with paintQuality% higher than 100 : {len(df_train[ df_train['paintQuality%'] > 100])}')
print(f' The proportion of observations with paintQuality% higher than 100 in the data set is : { round( (len(df_train[ df_train['paintQuality%'] > 100]) / len(df_train))*100, 2)} %')

 The number of observations with paintQuality% higher than 100 : 367
 The proportion of observations with paintQuality% higher than 100 in the data set is : 0.48 %


In [53]:
df_train.describe(include='object').T

,count,unique,top,freq
Brand,74452,72,Ford,14808
model,74456,735,Focus,6353
transmission,74451,40,Manual,38050
fuelType,74462,34,Petrol,37995


Almost half of the cars are Manual, and almost half are Petrol

In [54]:
df_train.isna().sum()

carID                0
Brand             1521
model             1517
year              1491
price                0
transmission      1522
mileage           1463
fuelType          1511
tax               7904
mpg               7926
engineSize        1516
paintQuality%     1524
previousOwners    1550
hasDamage         1548
dtype: int64

We have missing values in all the columns except for carID and price

In [55]:
df_train.duplicated().sum()

np.int64(0)

We have zero duplicates overall. We should also check if we have duplicates in the carID

In [56]:
df_train.duplicated(subset='carID').sum()

np.int64(0)

We also have 0 carID duplicates

In [52]:
df_train.groupby('Brand')['price'].mean().sort_values()

Brand
TOYOT        6690.000000
opel         9878.236364
OPEL        10084.710059
Opel        10368.416888
for         10560.857143
                ...     
mercedes    24820.666667
MERCEDE     27494.250000
mw          29447.500000
ERCEDES     31691.666667
ud          34646.666667
Name: price, Length: 72, dtype: float64

In [19]:
df_train['Brand'].value_counts(normalize=True, sort=True)

Brand
Ford        0.198893
Mercedes    0.144442
VW          0.131360
Opel        0.116115
BMW         0.093591
              ...   
OYOTA       0.000027
SKOD        0.000013
TOYOT       0.000013
skod        0.000013
toyot       0.000013
Name: proportion, Length: 72, dtype: float64

What I have to do:
- boxplots, histograms and barcharts for the variables
- list what we will have to do in the Data Preparation based on the Data Understanding results.